In [1]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam

In [2]:
data = pd.read_csv('IS_data.csv')
data

,Company,Time,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Price
0,CTAS,2019-05-01,6892.3,1147.9,885.0,64.97,8.25,6.17,2.05,1284.4,1147.9,654.6,221.83
1,PGR,2014-12-01,19377.3,2029.1,1281.0,32.81,2.17,2.02,0.69,2029.1,2029.1,1195.1,26.99
2,ELP,2018-12-01,3857.8,583.3,363.5,14.10,1.33,1.12,0.37,676.6,583.3,306.2,7.83
3,PHM,2016-12-01,7676.5,980.2,602.7,22.59,1.76,1.82,0.36,994.0,980.2,619.1,18.38
4,F,2019-12-01,155900.0,2658.0,47.0,39.25,0.01,0.40,0.60,2658.0,2658.0,1600.5,9.30
5,WM,2011-12-01,13378.0,2081.0,961.0,28.48,2.05,1.98,1.36,2132.0,2081.0,929.5,32.71
6,SIRI,2019-12-01,7794.0,1756.0,914.0,1.73,0.20,0.19,0.05,1897.0,1756.0,851.9,7.15
7,SCCO,2017-12-01,6654.5,2618.9,728.5,8.61,0.94,1.88,0.59,2624.8,2618.9,1449.7,46.81
8,BAX,2017-12-01,10584.0,1510.0,602.0,19.49,1.11,1.59,0.61,1664.0,1510.0,865.6,64.64
9,CTSH,2014-12-01,10263.0,1885.0,1439.0,16.88,2.37,1.98,0.00,1921.0,1885.0,1201.3,52.66


In [3]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)
data

Total Revenues
Done
Operating Income
Done
Net Income
Done
Revenue Per Share
Done
Basic EPS
Done
Normalized Basic EPS
Done
Dividend Per Share
Done
EBITA
Done
EBIT
Done
Normalized Net Income
Done


,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Price
0,0.013132,0.152721,0.191876,0.000414,0.000911,0.000826,0.155303,0.151308,0.152721,0.151562,221.83
1,0.036961,0.159877,0.195145,0.000208,0.000790,0.000720,0.052273,0.157298,0.159877,0.158824,26.99
2,0.007341,0.148136,0.187570,0.000089,0.000773,0.000697,0.028030,0.146419,0.148136,0.146882,7.83
3,0.014629,0.151359,0.189545,0.000143,0.000781,0.000715,0.027273,0.148972,0.151359,0.151086,18.38
4,0.297524,0.164984,0.184957,0.000249,0.000746,0.000678,0.045455,0.162357,0.164984,0.164270,9.30
5,0.025511,0.160299,0.192503,0.000181,0.000787,0.000719,0.103030,0.158126,0.160299,0.155255,32.71
6,0.014853,0.157659,0.192115,0.000010,0.000750,0.000673,0.003788,0.156236,0.157659,0.154213,7.15
7,0.012679,0.164667,0.190584,0.000054,0.000765,0.000716,0.044697,0.162090,0.164667,0.162244,46.81
8,0.020178,0.155661,0.189539,0.000123,0.000768,0.000709,0.046212,0.154362,0.155661,0.154397,64.64
9,0.019566,0.158707,0.196450,0.000107,0.000794,0.000719,0.000000,0.156429,0.158707,0.158907,52.66


In [4]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:10] # Parameters
y = NpMatrix[:,10] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

X: [[0.01313249 0.15272091 0.19187598 ... 0.15130826 0.15272091 0.15156246]
 [0.03696098 0.15987705 0.19514547 ... 0.15729821 0.15987705 0.1588236 ]
 [0.00734093 0.14813585 0.18757033 ... 0.14641947 0.14813585 0.14688201]
 ...
 [0.31257509 0.34565004 0.29170268 ... 0.38301227 0.34565004 0.30593264]
 [0.00518902 0.15225639 0.19026601 ... 0.14998432 0.15225639 0.15134751]
 [0.00816982 0.15014171 0.18915389 ... 0.147815   0.15014171 0.14926791]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 10
Total Number of Training instances: 5000


[221.83  26.99   7.83 ...  42.53  50.32  32.35]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 221.83
Total number of y values 5000


In [5]:
number_of_features = len(X[1])

#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=100, activation='relu', input_shape=(number_of_features,)))
    
        #Add Dropout Layer
    network.add(layers.Dropout(0.2))

    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [6]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size=5)

In [7]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/1000
4000/4000 [==============================] - 2s 571us/sample - loss: 56.9601 - mae: 56.9601 - mse: 23195.0762
Epoch 2/1000
4000/4000 [==============================] - 1s 344us/sample - loss: 47.4543 - mae: 47.4543 - mse: 21105.7344
Epoch 3/1000
4000/4000 [==============================] - 1s 344us/sample - loss: 47.0549 - mae: 47.0549 - mse: 21036.3691
Epoch 4/1000
4000/4000 [==============================] - 1s 290us/sample - loss: 46.6110 - mae: 46.6110 - mse: 21071.2930
Epoch 5/1000
4000/4000 [==============================] - 1s 322us/sample - loss: 46.2549 - mae: 46.2549 - mse: 20958.1484
Epoch 6/1000
4000/4000 [==============================] - 1s 335us/sample - loss: 45.7227 - mae: 45.7227 - mse: 20989.5078
Epoch 7/1000
4000/4000 [==============================] - 1s 334us/sample - loss: 45.4664 - mae: 45.4665 - mse: 20992.6543
Epoch 8/1000
4000/4000 [==============================] - 1s 310us/sample - loss: 45.0966 - mae: 45.0965 - mse: 20990

4000/4000 [==============================] - 1s 321us/sample - loss: 43.5333 - mae: 43.5333 - mse: 21120.3086
Epoch 67/1000
4000/4000 [==============================] - 1s 319us/sample - loss: 43.4878 - mae: 43.4878 - mse: 21115.4199
Epoch 68/1000
4000/4000 [==============================] - 1s 298us/sample - loss: 43.5553 - mae: 43.5553 - mse: 21122.3457
Epoch 69/1000
4000/4000 [==============================] - 1s 305us/sample - loss: 43.4425 - mae: 43.4424 - mse: 21144.3125
Epoch 70/1000
4000/4000 [==============================] - 1s 316us/sample - loss: 43.5651 - mae: 43.5651 - mse: 21134.6348
Epoch 71/1000
4000/4000 [==============================] - 1s 314us/sample - loss: 43.4665 - mae: 43.4665 - mse: 21136.3789
Epoch 72/1000
4000/4000 [==============================] - 1s 292us/sample - loss: 43.5363 - mae: 43.5363 - mse: 21134.0332
Epoch 73/1000
4000/4000 [==============================] - 1s 314us/sample - loss: 43.5708 - mae: 43.5708 - mse: 21143.1523
Epoch 74/1000
4000/400

4000/4000 [==============================] - 2s 391us/sample - loss: 43.4534 - mae: 43.4534 - mse: 21097.9609
Epoch 132/1000
4000/4000 [==============================] - 1s 367us/sample - loss: 43.3342 - mae: 43.3342 - mse: 21078.1074
Epoch 133/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 43.3598 - mae: 43.3598 - mse: 21126.4746
Epoch 134/1000
4000/4000 [==============================] - 1s 356us/sample - loss: 43.3017 - mae: 43.3017 - mse: 21041.1309
Epoch 135/1000
4000/4000 [==============================] - 2s 397us/sample - loss: 43.3828 - mae: 43.3828 - mse: 21101.7051
Epoch 136/1000
4000/4000 [==============================] - 1s 358us/sample - loss: 43.3899 - mae: 43.3899 - mse: 21081.4395
Epoch 137/1000
4000/4000 [==============================] - 1s 374us/sample - loss: 43.4589 - mae: 43.4589 - mse: 21130.3340
Epoch 138/1000
4000/4000 [==============================] - 1s 374us/sample - loss: 43.4247 - mae: 43.4246 - mse: 21085.4238
Epoch 139/1000


4000/4000 [==============================] - 2s 381us/sample - loss: 43.1960 - mae: 43.1960 - mse: 21040.3867
Epoch 197/1000
4000/4000 [==============================] - 1s 366us/sample - loss: 43.3549 - mae: 43.3549 - mse: 21083.5977
Epoch 198/1000
4000/4000 [==============================] - 1s 326us/sample - loss: 43.1556 - mae: 43.1556 - mse: 21021.2734
Epoch 199/1000
4000/4000 [==============================] - 1s 342us/sample - loss: 43.2760 - mae: 43.2759 - mse: 21044.6680
Epoch 200/1000
4000/4000 [==============================] - 1s 321us/sample - loss: 43.1414 - mae: 43.1415 - mse: 21062.2832
Epoch 201/1000
4000/4000 [==============================] - 2s 391us/sample - loss: 43.2057 - mae: 43.2057 - mse: 21019.5059
Epoch 202/1000
4000/4000 [==============================] - 1s 317us/sample - loss: 43.2255 - mae: 43.2255 - mse: 21038.2305
Epoch 203/1000
4000/4000 [==============================] - 1s 358us/sample - loss: 43.2256 - mae: 43.2256 - mse: 21021.7188
Epoch 204/1000


4000/4000 [==============================] - 1s 322us/sample - loss: 43.0814 - mae: 43.0813 - mse: 20984.0957
Epoch 262/1000
4000/4000 [==============================] - 1s 351us/sample - loss: 43.1413 - mae: 43.1413 - mse: 20990.7227
Epoch 263/1000
4000/4000 [==============================] - 1s 338us/sample - loss: 43.0369 - mae: 43.0369 - mse: 20954.5449
Epoch 264/1000
4000/4000 [==============================] - 1s 340us/sample - loss: 43.0431 - mae: 43.0431 - mse: 20951.1133
Epoch 265/1000
4000/4000 [==============================] - 1s 331us/sample - loss: 43.0740 - mae: 43.0740 - mse: 20989.0059
Epoch 266/1000
4000/4000 [==============================] - 1s 350us/sample - loss: 43.0192 - mae: 43.0192 - mse: 20932.8672
Epoch 267/1000
4000/4000 [==============================] - 1s 342us/sample - loss: 43.0858 - mae: 43.0858 - mse: 20975.6992
Epoch 268/1000
4000/4000 [==============================] - 1s 352us/sample - loss: 43.0488 - mae: 43.0488 - mse: 20959.9824
Epoch 269/1000


...

4000/4000 [==============================] - 1s 371us/sample - loss: 40.3825 - mae: 40.3825 - mse: 19885.5664
Epoch 717/1000
4000/4000 [==============================] - 2s 391us/sample - loss: 40.4390 - mae: 40.4390 - mse: 19840.6504
Epoch 718/1000
4000/4000 [==============================] - 1s 368us/sample - loss: 40.3730 - mae: 40.3730 - mse: 19856.8867
Epoch 719/1000
4000/4000 [==============================] - 2s 408us/sample - loss: 40.1835 - mae: 40.1835 - mse: 19860.3105
Epoch 720/1000
4000/4000 [==============================] - 1s 370us/sample - loss: 40.2072 - mae: 40.2072 - mse: 19832.5234
Epoch 721/1000
4000/4000 [==============================] - 2s 380us/sample - loss: 40.3573 - mae: 40.3573 - mse: 19863.1426
Epoch 722/1000
4000/4000 [==============================] - 2s 383us/sample - loss: 40.2747 - mae: 40.2747 - mse: 19858.0879
Epoch 723/1000
4000/4000 [==============================] - 2s 387us/sample - loss: 40.3874 - mae: 40.3874 - mse: 19900.8418
Epoch 724/1000


4000/4000 [==============================] - 2s 392us/sample - loss: 40.1100 - mae: 40.1100 - mse: 19783.6602
Epoch 782/1000
4000/4000 [==============================] - 1s 364us/sample - loss: 40.1404 - mae: 40.1404 - mse: 19790.3926
Epoch 783/1000
4000/4000 [==============================] - 2s 405us/sample - loss: 39.9674 - mae: 39.9674 - mse: 19755.4902
Epoch 784/1000
4000/4000 [==============================] - 2s 389us/sample - loss: 40.0702 - mae: 40.0702 - mse: 19767.4238
Epoch 785/1000
4000/4000 [==============================] - 2s 377us/sample - loss: 40.0776 - mae: 40.0776 - mse: 19819.1660
Epoch 786/1000
4000/4000 [==============================] - 1s 368us/sample - loss: 40.0307 - mae: 40.0307 - mse: 19749.3535
Epoch 787/1000
4000/4000 [==============================] - 2s 406us/sample - loss: 40.2107 - mae: 40.2107 - mse: 19803.7148
Epoch 788/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 40.1400 - mae: 40.1399 - mse: 19854.3066
Epoch 789/1000


4000/4000 [==============================] - 1s 372us/sample - loss: 40.0176 - mae: 40.0176 - mse: 19794.5117
Epoch 847/1000
4000/4000 [==============================] - 2s 400us/sample - loss: 40.0329 - mae: 40.0329 - mse: 19717.9062
Epoch 848/1000
4000/4000 [==============================] - 2s 391us/sample - loss: 39.9105 - mae: 39.9105 - mse: 19689.2520
Epoch 849/1000
4000/4000 [==============================] - 1s 366us/sample - loss: 39.9827 - mae: 39.9827 - mse: 19770.8770
Epoch 850/1000
4000/4000 [==============================] - 1s 372us/sample - loss: 40.0539 - mae: 40.0539 - mse: 19799.3926
Epoch 851/1000
4000/4000 [==============================] - 2s 406us/sample - loss: 40.1790 - mae: 40.1790 - mse: 19817.0488
Epoch 852/1000
4000/4000 [==============================] - 2s 383us/sample - loss: 40.1097 - mae: 40.1097 - mse: 19765.9727
Epoch 853/1000
4000/4000 [==============================] - 2s 378us/sample - loss: 39.9581 - mae: 39.9581 - mse: 19760.9629
Epoch 854/1000


4000/4000 [==============================] - 2s 388us/sample - loss: 39.8377 - mae: 39.8377 - mse: 19709.3301
Epoch 912/1000
4000/4000 [==============================] - 2s 405us/sample - loss: 40.0257 - mae: 40.0257 - mse: 19699.4355- loss: 51.5055 -
Epoch 913/1000
4000/4000 [==============================] - 1s 368us/sample - loss: 39.9665 - mae: 39.9664 - mse: 19745.4316
Epoch 914/1000
4000/4000 [==============================] - 1s 338us/sample - loss: 39.6479 - mae: 39.6479 - mse: 19636.7930
Epoch 915/1000
4000/4000 [==============================] - 1s 331us/sample - loss: 39.9783 - mae: 39.9783 - mse: 19657.7109
Epoch 916/1000
4000/4000 [==============================] - 1s 297us/sample - loss: 39.6394 - mae: 39.6393 - mse: 19736.0508
Epoch 917/1000
4000/4000 [==============================] - 1s 297us/sample - loss: 40.1696 - mae: 40.1696 - mse: 19762.3613
Epoch 918/1000
4000/4000 [==============================] - 1s 299us/sample - loss: 39.8112 - mae: 39.8112 - mse: 19718.150

4000/4000 [==============================] - 1s 297us/sample - loss: 39.7016 - mae: 39.7016 - mse: 19622.1953
Epoch 977/1000
4000/4000 [==============================] - 1s 293us/sample - loss: 39.9166 - mae: 39.9167 - mse: 19690.0215
Epoch 978/1000
4000/4000 [==============================] - 1s 315us/sample - loss: 39.8758 - mae: 39.8758 - mse: 19696.7617
Epoch 979/1000
4000/4000 [==============================] - 1s 295us/sample - loss: 40.0121 - mae: 40.0121 - mse: 19676.1387
Epoch 980/1000
4000/4000 [==============================] - 1s 296us/sample - loss: 39.6051 - mae: 39.6051 - mse: 19653.0820
Epoch 981/1000
4000/4000 [==============================] - 1s 296us/sample - loss: 39.5567 - mae: 39.5567 - mse: 19676.3184
Epoch 982/1000
4000/4000 [==============================] - 1s 337us/sample - loss: 39.6769 - mae: 39.6769 - mse: 19661.6875
Epoch 983/1000
4000/4000 [==============================] - 1s 297us/sample - loss: 39.5593 - mae: 39.5593 - mse: 19633.7930
Epoch 984/1000


In [8]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-34.67152086 -51.36645475 -32.86812697 -36.12697398 -36.80591664]
Mean MAE: -38.367798640728
Standard Deviation of the mean: 6.638293290072823
